<a href="https://colab.research.google.com/github/AaronWoodhouse/DiseasePrediction/blob/main/MLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import (
    DataLoader,
    TensorDataset,
    Dataset,
    random_split
)
#import torchvision
from torchsummaryX import summary
import numpy as np
import pandas as pd
#import os
import matplotlib.pyplot as plt
from importlib import reload
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [ ]:
# make df into numbers, can't handle strings
# bit map of possible symptoms, class labels mapped to index of labels list
 
df = pd.read_csv("dataset.csv")

labels = df["Disease"]
label_map = df["Disease"].unique() # unique class labels
df = df.drop('Disease', axis=1)

# replace labels by class index
for i, x in enumerate(label_map):
    indicies = np.where(labels == x)[0]
    labels.loc[indicies] = i

# all unique symptoms
features = np.array([])
for col in df:
    features = np.concatenate((features, df[col].unique()))

# dataframe of zeros, with columns as each symptom
new_df = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=features)
new_df = new_df.drop(np.nan, axis=1) # remove nan

# get list of symptoms for each row
strings = []
for i in range(0, df.shape[0]):
    strings.append([x for x in df.iloc[i] if x == x])

# make bitmap
for i, symptoms in enumerate(strings): # add labels
    for x in symptoms:
        new_df.loc[i, x] = 1

# add labels to new_df
new_df.insert(0, "Disease", labels)
df = new_df
original_df = df

In [ ]:
# DEBUGGING #
# clears all variables
#%reset

In [ ]:
class DiseaseDataset(Dataset):
 
  def __init__(self,DataFrame): 
    x = torch.tensor(df.drop('Disease', axis=1).values)
    y = torch.tensor(df["Disease"].values.astype(float))
 
    self.x_train=torch.tensor(x, dtype=torch.float32)
    self.y_train=torch.tensor(y, dtype=torch.int64)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx], self.y_train[idx]

In [ ]:
df = DiseaseDataset(df)

In [ ]:
train_dataset, val_dataset = random_split(df, (0.6, 0.4))
train_dataloader = DataLoader(train_dataset, batch_size=30, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

In [ ]:
class LinearClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.LazyLinear(41),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
from torch.optim import (Optimizer, Adam)
from torch.nn.functional import cross_entropy
from torchmetrics import Accuracy

In [ ]:
import trainer_lib

In [ ]:
m = LinearClassifier()
xs, ys = next(iter(train_dataloader))
summary(m, xs);

                     Kernel Shape Output Shape  Params  Mult-Adds
Layer                                                            
0_model.LazyLinear_0    [394, 41]     [30, 41]   16195      16154
-----------------------------------------------------------------
                      Totals
Total params           16195
Trainable params       16195
Non-trainable params       0
Mult-Adds              16154


In [ ]:
reload(trainer_lib)

model_linear = LinearClassifier()
trainer = trainer_lib.Trainer(model_linear, train_dataloader, val_dataloader)
trainer.reset()
model_linear_log = trainer.train(epochs=20, max_batches=10)
model_linear_log.round(2)

[0 (0.05s)]: train_loss=3.57 train_acc=0.15, val_loss=3.38 val_acc=0.43
[1 (0.05s)]: train_loss=3.18 train_acc=0.56, val_loss=3.02 val_acc=0.56
[2 (0.04s)]: train_loss=2.86 train_acc=0.64, val_loss=2.70 val_acc=0.71
[3 (0.04s)]: train_loss=2.60 train_acc=0.81, val_loss=2.42 val_acc=0.86
[4 (0.06s)]: train_loss=2.36 train_acc=0.91, val_loss=2.18 val_acc=0.94
[5 (0.04s)]: train_loss=2.06 train_acc=0.98, val_loss=1.97 val_acc=0.97
[6 (0.04s)]: train_loss=1.88 train_acc=0.97, val_loss=1.80 val_acc=0.97
[7 (0.05s)]: train_loss=1.69 train_acc=0.97, val_loss=1.64 val_acc=0.99
[8 (0.05s)]: train_loss=1.60 train_acc=0.98, val_loss=1.51 val_acc=0.98
[9 (0.04s)]: train_loss=1.56 train_acc=1.00, val_loss=1.39 val_acc=0.99
[10 (0.04s)]: train_loss=1.31 train_acc=1.00, val_loss=1.28 val_acc=1.00
[11 (0.04s)]: train_loss=1.14 train_acc=1.00, val_loss=1.19 val_acc=1.00
[12 (0.04s)]: train_loss=1.16 train_acc=1.00, val_loss=1.11 val_acc=1.00
[13 (0.04s)]: train_loss=1.03 train_acc=1.00, val_loss=1.03 v

,train_loss,train_accuracy,val_loss,val_accuracy,epoch_duration
0,3.57,0.15,3.38,0.43,0.05
1,3.18,0.56,3.02,0.56,0.05
2,2.86,0.64,2.70,0.71,0.04
3,2.60,0.81,2.42,0.86,0.04
4,2.36,0.91,2.18,0.94,0.06
5,2.06,0.98,1.97,0.97,0.04
6,1.88,0.97,1.80,0.97,0.04
7,1.69,0.97,1.64,0.99,0.05
8,1.60,0.98,1.51,0.98,0.05
9,1.56,1.00,1.39,0.99,0.04


In [ ]:
data = torch.tensor(original_df.drop('Disease', axis=1).values)
data = torch.tensor(data, dtype=torch.float32)

# data samples to use (a->b-1)
a = 3000
b = 3051

with torch.no_grad():
    model_linear.eval()

    for i in range(a,b): #len(labels)
        index = i
        true_label = labels[index]
        test_sample = data[index]
        output = model_linear(test_sample)

        sm = nn.Softmax(dim=0)
        probabilities = sm(output)

        pred = np.argmax(output)
        print(f'Prediction: {pred} - Actual target: {true_label} - Confidence: {probabilities[pred] * 100}')
        print(f"Probabilities (%): \n{probabilities * 100}")
        print()

    model_linear.train()

Prediction: 7 - Actual target: 7 - Confidence: 87.18656921386719
Probabilities (%): 
tensor([ 0.2698,  0.1888,  0.1827,  0.1451,  0.2968,  0.2638,  0.2405, 87.1866,
         0.2283,  0.2486,  0.1914,  0.4770,  0.3001,  0.2540,  0.6781,  0.1166,
         0.6320,  0.1483,  0.2186,  0.1351,  0.4596,  0.3190,  0.3029,  0.2417,
         0.1806,  0.2924,  0.2521,  0.2278,  0.2353,  0.2520,  1.1173,  0.5077,
         1.0178,  0.6014,  0.2312,  0.2112,  0.2024,  0.2188,  0.2011,  0.2625,
         0.2632])

Prediction: 8 - Actual target: 8 - Confidence: 33.1131591796875
Probabilities (%): 
tensor([ 1.2838,  0.9847,  1.6983,  1.4444,  1.3061,  2.7132,  1.3230,  1.1112,
        33.1132,  1.2646,  1.0403,  1.2322,  1.5603,  2.0549,  1.8530,  5.2949,
         1.2966,  1.5240,  4.9296,  4.0155,  0.9606,  0.7850,  1.1533,  1.1181,
         1.9933,  1.3878,  0.8430,  0.8654,  1.1997,  1.9465,  1.1817,  1.0349,
         3.0522,  1.7453,  1.5179,  1.1750,  2.1171,  1.1630,  1.3079,  1.2877,
         1.1